In [ ]:
import os
import sys

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)


In [ ]:
import pandas as pd
from DateProvider import DateProvider
from datetime import datetime
from ColumnsAdder import ColumnsAdder
from MedianOfFreeBedsByKreisTableFactory import MedianOfFreeBedsByKreisTableFactory
from TimeseriesReader import readTimeseries
from IntensivstationenHtmlFileUpdater import saveLastUpdatedIntensivstationen, saveKreisOptions
from KreiseReader import readKreise
from KreisOptionsProvider import getKreisOptionsAndAlleLandkreise
from IntensiveCareBedsPersister import getAndPersistIntensiveCareBeds4AlleKreise
from Chart import createMedianOfFreeBedsByKreisChart


pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', None)
pd.set_option('mode.chained_assignment', 'raise')


In [ ]:
print(datetime.now().strftime("%d.%m.%Y, %H:%M:%S Uhr"))

In [ ]:
dateProvider = DateProvider()
print('          lastUpdated:', dateProvider.getLastUpdated())
print('lastUpdatedDataSource:', dateProvider.getLastUpdatedDataSource())        
needsUpdate = True # dateProvider.needsUpdate()
print('needsUpdate:', needsUpdate)

In [ ]:
saveLastUpdatedIntensivstationen(
    dateProvider.getLastUpdatedDataSource(),
    toHtmlFile = "../../docs/intensivstationen.html")

In [ ]:
timeSeries = readTimeseries(download = needsUpdate)
timeSeries

In [ ]:
kreisValues = sorted(timeSeries['Kreis'].drop_duplicates().values)
saveKreisOptions(
    kreisOptions = getKreisOptionsAndAlleLandkreise(kreisValues),
    toHtmlFile = "../../docs/intensivstationen.html")

In [ ]:
getAndPersistIntensiveCareBeds4AlleKreise(
    timeSeries,
    intensivstationenDataDir = '../../docs/data/intensivstationen',
    kreisValues = kreisValues)


In [ ]:
medianOfFreeBedsByKreisTable = MedianOfFreeBedsByKreisTableFactory(timeSeries).createMedianOfFreeBedsByKreisTable('Kreis')
medianOfFreeBedsByKreisTable.reset_index().to_json(
    '../../docs/data/intensivstationen/medianOfFreeBedsByKreisTable.json',
    orient = "records")

In [ ]:
def getChartTitle(dateStart, dateEnd):
    def formatDate(date):
        return date.strftime("%d.%m.%Y")

    return f"Median freier Intensivbetten im Zeitraum {formatDate(dateStart)} bis {formatDate(dateEnd)}"

In [ ]:
createMedianOfFreeBedsByKreisChart(
    timeSeries,
    chartTitle = getChartTitle(
        dateStart=timeSeries['date'].min(),
        dateEnd=timeSeries['date'].max()))